In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('hvfhv_taxi') \
    .getOrCreate()

22/03/01 11:45:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [6]:
df_green.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [3]:
df_green.registerTempTable('trips_data')

In [17]:
df_grenn_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,    

    -- Revenue calculation 
    ROUND(SUM(total_amount), 2) AS amount,
    COUNT(*) AS nr_records
    
FROM trips_data
WHERE lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY 1, 2
--ORDER BY 4 DESC;
""")

In [19]:
df_grenn_revenue.write.parquet('data/report/revenue/green', mode = 'overwrite')